**CÀI ĐẶT THƯ VIỆN**

In [ ]:
#Thư viện xử lý ảnh
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import cv2

#Thư viện xử lý dữ liệu
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import numpy as np

#Thư viện tùy chỉnh mô hình
from tensorflow.keras.layers import Flatten,Dropout,Dense,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

#Thư viên đồ thị
import matplotlib.pyplot as plt
# import pickle
# import os

**ĐƯỜNG DẪN THƯ MỤC THIẾT LẬP**

In [ ]:
CSV_Path='/kaggle/input/brain-tumor/Brain_11_6/_info.csv'
Image_Path='/kaggle/input/brain-tumor/Brain_11_6/'
MaxImage=5000
MyEpoch=200#Vì dữ liệu khá lớn nên việc học nhiều giúp học được tốt từ dữ liệu đa dạng.
#Nếu tập dữ liệu nhỏ, việc chạy quá nhiều epochs có thể dẫn đến hiện tượng overfitting
#(mô hình quá tinh chỉnh cho tập dữ liệu huấn luyện mà không tổng quát hóa tốt cho dữ liệu mới).
MyBatch=16#Tập dữ liệu khá lớn nên cần chọn bath size lớn tùy vào tập dữ liệu và bộ nhớ thiết bị
#Một số nghiên cứu đã chỉ ra rằng batch size lớn có thể đạt được độ chính xác tốt hơn
#và ổn định hơn trong quá trình huấn luyện.

**ĐỌC ẢNH VÀ TIỀN XỬ LÝ ẢNH**

In [ ]:
Dataset=pd.read_csv(CSV_Path)
filenames=Dataset.iloc[:MaxImage,:-5].values
labels=Dataset.iloc[:MaxImage,1:2].values
bboxes=Dataset.iloc[:MaxImage,2:].values

#Xóa biến không xử dụng nữa để giải phóng vùng nhớ
del Dataset

data = []
for filename in filenames:
    imagePath=Image_Path+filename[0]
    #Đọc ảnh và đưa ảnh về kích thước 224x224
    image=load_img(imagePath,target_size=(224, 224))
    #Chuyển ảnh sang dạng mảng(ma trận) sau đó đưa vào biến data giữ
    image = img_to_array(image)
    data.append(image)

In [ ]:
#Vì ma trận ảnh về kiểu dữ liệu float32 trong khoản 0-1 để có hiệu xuất tốt hơn với 1 số lý do:
# *Tính toán dễ dàng chính xác hơn, các phép tính tích chập, gradient được tối ưu.
# *Giúp mô hình dễ dàng học thông tin quan trọng. Trong khi ma trận cần phải được chuẩn hóa lại.
# *Tối ưu hóa phần cứng vì 1 số GPU tính toán tốt trên kiểu dữ liệu float32
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
#Do đưa ảnh về float32 và vì toàn bộ ảnh đều có kích thước 640x640 nên có thể chia trực tiếp
#trên mảng để lấy ra tọa độ box trên ma trận ảnh lúc này là kiểu float32
bboxes = np.array(bboxes, dtype="float32") / 640.0
#Vì label là chữ nên cần mã hóa thành dạng(one-hot vector) để dễ xử lý hơn
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

**TÁCH TẬP DỮ LIỆU**

In [ ]:
#Tách tập dữ liệu ra thành tập dữ liệu train(80%) và val(20%)
#Tập val được sử dụng để đánh giá hiệu suất của mô hình trên dữ liệu không được sử dụng
#trong quá trình huấn luyện, giúp đánh giá mức độ tổng quát hóa của mô hình.
split = train_test_split(data, labels, bboxes,test_size=0.20, random_state=42)
(trainImages, valImages) = split[:2]
(trainLabels, valLabels) = split[2:4]
(trainBBoxes, valBBoxes) = split[4:]

#Xóa biến không xử dụng nữa để giải phóng vùng nhớ
del data,bboxes,filenames,split

**CÀI ĐẶT THƯ VIỆN VÀ THIẾT LẬP MODEL**

In [ ]:
from tensorflow.keras.applications import ResNet101
# from tensorflow.keras.applications import InceptionV3
# from tensorflow.keras.applications import MobileNet
# from tensorflow.keras.applications import DenseNet121

In [ ]:
#Tạo model input thuộc loại phân loại hình ảnh weights="imagenet"
#include_top=False loại bỏ layer đầu
#Đầu vào của model có kích thước 224x224 3(số lớp => ảnh màu)
Model_input = ResNet101(weights="imagenet", 
                        include_top=False,
                        input_tensor=Input(shape=(224, 224, 3)))

# Model_input = InceptionV3(weights="imagenet", 
#                         include_top=False,
#                         input_tensor=Input(shape=(224, 224, 3)))

# Model_input = MobileNet(weights="imagenet", 
#                         include_top=False,
#                         input_tensor=Input(shape=(224, 224, 3)))

# Model_input = DenseNet121(weights="imagenet", 
#                         include_top=False,
#                         input_tensor=Input(shape=(224, 224, 3)))

# Lấy tên model để lưu tập tin
Aplication_Name=model.name
history_file = Aplication_Name + "_history.csv"

#Tắt các trọng số của Model_Input
Model_input.trainable = False
flatten = Model_input.output

#Flatten(làm phẳng) biến đổi đầu vào từ 3D thành 1D
flatten = Flatten()(flatten)

**THAY ĐỔI KÍCH THƯỚC ĐẦU RA**

In [ ]:
#relu mô phỏng những nơ ron có tỷ lệ truyền xung qua axon
#sigmoid sự thay đổi nhỏ trong input dẫn đến một kết quả output ko mấy thay đổi.

#Lấy thông tin của ảnh khi được truyền vào
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
#Lấy thông tin của ảnh khi được truyền vào xử dụng để đoán tọa độ box
bboxHead = Dense(4, activation="sigmoid",name="bounding_box")(bboxHead)

#Lấy thông tin của ảnh khi được truyền vào(flatten)
softmaxHead = Dense(512, activation="relu")(flatten)
#Lớp này giúp ngăn chặn việc overfitting bằng cách tắt ngẫu nhiên 50% đơn vị(512)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
#Lấy thông tin của ảnh khi được truyền vào xử dụng để đoán nhãn của đối tượng
softmaxHead = Dense(len(lb.classes_), activation="softmax",name="class_label")(softmaxHead)

model = Model(
    inputs=Model_input.input,
    outputs=(bboxHead, softmaxHead))

In [ ]:
losses = {
    #Định nghĩa "categorical_crossentropy" thích hợp cho dữ liệu dạng one-hot vector
    "class_label": "categorical_crossentropy",
    #Định nghĩa "mean_squared_error" do lường sai số trung bình của tọa độ box
    "bounding_box": "mean_squared_error",
}
#Định nghĩa độ quan trọng của loss
lossWeights = {
    "class_label": 1.0,
    "bounding_box": 1.0
}

**THIẾT LẬP TÙY CHỌN CẢI TIẾN CHO MODEL**

In [ ]:
import keras.backend as K
import tensorflow as tf

def sparse_precision(y_true, y_pred):
    #K.clip Giới hạn lại trong khoản 0-1
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def val_sparse_precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def sparse_recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def val_sparse_recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def sparse_f1(y_true, y_pred):
    precision = sparse_precision(y_true, y_pred)
    recall = sparse_recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
    
def val_sparse_f1(y_true, y_pred):
    precision = val_sparse_precision(y_true, y_pred)
    recall = val_sparse_recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

train_auc_metric = tf.keras.metrics.AUC(name='train_auc')
val_auc_metric = tf.keras.metrics.AUC(name='val_auc')
@tf.function
def sparse_auc(y_true, y_pred):
    train_auc_metric.update_state(y_true, y_pred)
    return train_auc_metric.result()

@tf.function
def val_sparse_auc(y_true, y_pred):
    val_auc_metric.update_state(y_true, y_pred)
    return val_auc_metric.result()

# Cài metric tùy chỉnh
import keras.metrics
keras.metrics.sparse_precision = sparse_precision
keras.metrics.val_sparse_precision = val_sparse_precision
keras.metrics.sparse_recall = sparse_recall
keras.metrics.val_sparse_recall = val_sparse_recall
keras.metrics.sparse_f1 = sparse_f1
keras.metrics.val_sparse_f1 = val_sparse_f1
keras.metrics.sparse_auc = sparse_auc
keras.metrics.val_sparse_auc = val_sparse_auc

In [ ]:
#Điều chỉnh tốc độ học của mô hình với, Adam tự động điều chỉnh mức độ học tùy thuộc
#vào sự thay đổi của từng tham số trong quá trình huấn luyện
opt = Adam(learning_rate=0.0001)

#Cấu hình mô hình để sử dụng các thông số cần thiết cho quá trình huấn luyện.
model.compile(loss=losses, optimizer=opt, metrics=["accuracy", "sparse_precision", "val_sparse_precision", "sparse_recall",
                                                   "val_sparse_recall","sparse_f1","val_sparse_f1","sparse_auc", "val_sparse_auc"], loss_weights=lossWeights)

In [ ]:
#Định nghĩa lại cho trainTargets và valTargets
trainTargets = {
    "class_label": trainLabels,
    "bounding_box": trainBBoxes
}
valTargets = {
    "class_label": valLabels,
    "bounding_box": valBBoxes
}

**HUẤN LUYỆN MODEL**

In [ ]:
H = model.fit(
    trainImages, trainTargets,
    validation_data=(valImages, valTargets),
    batch_size=MyBatch,
    epochs=MyEpoch,
    verbose=2)
#verbose hiển thị thông tin huấn luyện chi tiết của mỗi epoch

**LƯU MODEL**

In [ ]:
model.save('/kaggle/working/'+Aplication_Name+'.h5')

In [ ]:
pd.DataFrame.from_dict(H.history).to_csv(Aplication_Name+'_history.csv',index=False)

**Accuracy**

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(H.history['class_label_accuracy'], label='Train Accuracy')
plt.plot(H.history['val_class_label_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Vẽ biểu đồ Loss
plt.subplot(1, 2, 2)
plt.plot(H.history['class_label_loss'], label='Train Loss')
plt.plot(H.history['val_class_label_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
plt.figure(figsize=(15, 5))

# Vẽ biểu đồ Precision
plt.subplot(2, 2, 1)  
plt.plot(H.history['class_label_sparse_precision'], label='Train Precision')
plt.plot(H.history['val_class_label_sparse_precision'], label='Validation Precision')
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.legend()

# Vẽ biểu đồ Recall
plt.subplot(2, 2, 2)  
plt.plot(H.history['class_label_val_sparse_recall'], label='Train Recall')
plt.plot(H.history['val_class_label_val_sparse_recall'], label='Validation Recall')
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.legend()

# Vẽ biểu đồ F1-score
plt.subplot(2, 2, 3)
plt.plot(H.history['class_label_sparse_f1'], label='Train F1-score')
plt.plot(H.history['val_class_label_sparse_f1'], label='Validation F1-score')
plt.xlabel('Epochs')
plt.ylabel('F1-score')
plt.legend()

# Vẽ biểu đồ AUC
plt.subplot(2, 2, 4)
plt.plot(H.history['class_label_sparse_auc'], label='Train AUC')
plt.plot(H.history['val_class_label_sparse_auc'], label='Validation AUC')
plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.legend()

plt.tight_layout()
plt.show()

**ĐỒ THỊ ACCURACY VÀ LOSS CỦA MODEL**

In [ ]:
# N = np.arange(0, 500)
# plt.style.use("ggplot")
# (fig, ax) = plt.subplots(1, 2, figsize=(24, 4))

# acc=H.history["class_label_accuracy"]
# val_acc=H.history["val_class_label_accuracy"]

# ax[0].plot(N,acc,label="label_accuracy",color='crimson')
# ax[0].plot(N,val_acc,label="val_label_accuracy",color='salmon')
# ax[0].set_xlabel("Epoch")
# ax[0].set_ylabel("Accuracy")
# ax[0].legend()

# loss=H.history["class_label_loss"]
# val_loss=H.history["val_class_label_loss"]

# ax[1].plot(N,loss,label="label_loss",color='crimson')
# ax[1].plot(N,val_loss,label="val_label_loss",color='salmon')
# ax[1].set_xlabel("Epoch")
# ax[1].set_ylabel("Loss")
# ax[1].legend()

**In các thông số kiểm thử sau huấn luyện**

In [ ]:
# Giả sử testImages và testTargets là dữ liệu kiểm thử của bạn
test_loss = model.evaluate(valImages, valTargets, verbose=0)

metrics = ['loss', 'bounding_box_loss', 'class_label_loss', 'bounding_box_accuracy', 'bounding_box_sparse_precision', 'bounding_box_val_sparse_precision', 'bounding_box_sparse_recall', 'bounding_box_val_sparse_recall', 'bounding_box_sparse_auc', 'bounding_box_val_sparse_auc', 'class_label_accuracy', 'class_label_sparse_precision', 'class_label_val_sparse_precision', 'class_label_sparse_recall', 'class_label_val_sparse_recall', 'class_label_sparse_auc', 'class_label_val_sparse_auc', 'val_loss', 'val_bounding_box_loss', 'val_class_label_loss', 'val_bounding_box_accuracy', 'val_bounding_box_sparse_precision', 'val_bounding_box_val_sparse_precision', 'val_bounding_box_sparse_recall', 'val_bounding_box_val_sparse_recall', 'val_bounding_box_sparse_auc', 'val_bounding_box_val_sparse_auc', 'val_class_label_accuracy', 'val_class_label_sparse_precision', 'val_class_label_val_sparse_precision', 'val_class_label_sparse_recall', 'val_class_label_val_sparse_recall', 'val_class_label_sparse_auc', 'val_class_label_val_sparse_auc']

for metric, value in zip(metrics, test_loss):
    print(f"{metric}: {value:.4f}")

In [ ]:
# del trainLabels, valLabels
# del trainBBoxes, valBBoxes

**DỰ ĐOÁN ĐỂ HIỂN THỊ MA TRẬN NHẰM LẪN**

In [ ]:
(boxPreds, labelPreds) = model.predict(trainImages)

In [ ]:
labelPreds=(np.rint(labelPreds)).astype(int)
labelPreds=np.array(labelPreds, dtype="float32")
labelPreds=np.argmax(labelPreds,axis=1)
labelTargets=trainTargets['class_label']
labelTargets=np.argmax(labelTargets,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
confusion_mtx = confusion_matrix(labelTargets, labelPreds)
confusion_mtx_percent = confusion_mtx.astype('float') / confusion_mtx.sum(axis=1)[:, np.newaxis]
f,ax = plt.subplots(figsize=(8,5))
sns.heatmap(confusion_mtx_percent, annot=True, fmt=".2%",cmap='Blues')
ax.xaxis.set_ticklabels(['normal', 'tumor']);
ax.yaxis.set_ticklabels(['normal', 'tumor']);

**HIỂN THỊ ĐỒ THỊ ROC-AUC**

In [ ]:
ROC_AUC_Path='/kaggle/working/'+Aplication_Name+'.png'

from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
import matplotlib.pyplot as plt
from itertools import cycle

n_classes = 2

# Dự đoán trên tập validation
(temp,y_score) = model.predict(valImages)

# Binarize the output
y_test_bin = label_binarize(valTargets['class_label'], classes=[0, 1])

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='Đường cong trung bình ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]))

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color,
             label='ROC curve của lớp {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Đường cong dự đoán của não trên 2 lớp')
plt.legend(loc="lower right")
plt.savefig(ROC_AUC_Path)
plt.show()

**DỰ ĐOÁN ẢNH**

In [ ]:
!pip install imutils

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
import cv2
import math
import random
import imutils
import numpy as np
import matplotlib.pyplot as plt

# model = load_model('/kaggle/input/fileh5-1/DenseNet121.h5')
# model = load_model('/kaggle/input/fileh5-2/InceptionV3.h5')
# model = load_model('/kaggle/input/fileh5-1/MobileNet.h5')
model = load_model('/kaggle/input/fileh5-1/DenseNet121.h5')


img_num = 3

def get_random_image_files(directory, max_images=img_num):
    image_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                image_files.append(os.path.join(root, file))
    return random.sample(image_files, min(max_images, len(image_files)))

image_files = get_random_image_files('/kaggle/input/brain-segmentation-for-healthy-and-tumor/test')


# image_files = ['/kaggle/input/brain-segmentation-for-healthy-and-tumor/test/Not-Cancer-1006-_jpg.rf.e3bc481c06f757b92ff43c229081709c.jpg',
#                '/kaggle/input/brain-segmentation-for-healthy-and-tumor/test/Not-Cancer-1018-_jpg.rf.e60c493960c53fb4907d1aad0aab2bd9.jpg',
#                '/kaggle/input/brain-segmentation-for-healthy-and-tumor/test/Not-Cancer-1049-_jpg.rf.51c3dc26156da4b80f48d9d4cc2acd88.jpg']


# Tính số lượng hàng và cột cho subplot
n = len(image_files)
cols = math.ceil(np.sqrt(n))
rows = math.ceil(n / cols)

plt.figure(figsize=(30, 10))  # Adjust as needed

for idx, image_file in enumerate(image_files):
    image = load_img(image_file, target_size=(224, 224))
    print(str(idx)+" -- "+str(image_file))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    (boxPreds, labelPreds) = model.predict(image)
    i = np.argmax(labelPreds, axis=1)
    label = 'tumor' if i else 'normal'

    size_label = "N/A"
    if label=='tumor':
        # Lấy ví trí box của ảnh
        (startX, startY, endX, endY) = boxPreds[0]
        image = cv2.imread(image_file)
        image = imutils.resize(image, width=600)
        (h, w) = image.shape[:2]
        startX = int(startX * w)
        startY = int(startY * h)
        endX = int(endX * w)
        endY = int(endY * h)
        y = startY - 10 if startY - 10 > 10 else startY + 10

        # Tính toán kích thước khối u
        tumor_width = endX - startX
        tumor_height = endY - startY

        # Quyết định đơn vị dựa trên kích thước
        size_label = f"{tumor_width}mm x {tumor_height}mm"
        if tumor_width >= 10 and tumor_height >= 10:
            tumor_width /= 10
            tumor_height /= 10
            size_label = f"{tumor_width}mm x {tumor_height}mm"

        # In lớp vào ảnh
        cv2.putText(image, f"{label}", (startX, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 255), 2)
        
        # Vẽ box
        cv2.rectangle(image, (startX, startY), (endX, endY),(0, 0, 255), 2)
    else:
        image = cv2.imread(image_file)
        image = imutils.resize(image, width=600)
        cv2.putText(image, label, (20,20), cv2.FONT_HERSHEY_SIMPLEX,0.7, (0, 255, 0), 2)

    plt.subplot(rows, cols, idx+1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    # In lớp vào ảnh và đặt ở ví trí góc trái bên ảnh
    plt.title(f'{label}: {np.max(labelPreds) * 100:.2f}%, size: {size_label}')
    plt.axis('off')
plt.show()